In [0]:
-- ----------------------------------------
-- 1. DROP TABLES (to allow script rerun)
-- ----------------------------------------

DROP TABLE IF EXISTS admissions;
DROP TABLE IF EXISTS patients;
DROP TABLE IF EXISTS doctors;
DROP TABLE IF EXISTS province_names;


-- ----------------------------------------
-- 2. CREATE TABLES
-- ----------------------------------------

-- Table: province_names (Reference Data)
CREATE TABLE province_names (
    province_id CHAR(2) PRIMARY KEY,
    province_name VARCHAR(30) NOT NULL
);

-- Table: doctors
CREATE TABLE doctors (
    doctor_id BIGINT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    first_name VARCHAR(30) NOT NULL,
    last_name VARCHAR(30) NOT NULL,
    specialty VARCHAR(25)
);

-- Table: patients
CREATE TABLE patients (
    patient_id BIGINT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    first_name VARCHAR(30) NOT NULL,
    last_name VARCHAR(30) NOT NULL,
    gender CHAR(1),
    birth_date DATE,
    city VARCHAR(30),
    province_id CHAR(2),
    allergies VARCHAR(80),
    height DECIMAL(3,0),
    weight DECIMAL(4,0),
    FOREIGN KEY (province_id) REFERENCES province_names(province_id)
);

-- Table: admissions
CREATE TABLE admissions (
    admission_id BIGINT GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    patient_id BIGINT NOT NULL,
    admission_date DATE NOT NULL,
    discharge_date DATE,
    diagnosis VARCHAR(50),
    attending_doctor_id BIGINT,
    FOREIGN KEY (patient_id) REFERENCES patients(patient_id),
    FOREIGN KEY (attending_doctor_id) REFERENCES doctors(doctor_id)
);


-- ----------------------------------------
-- 3. INSERT INTO TABLES (Sample Data)
-- ----------------------------------------

-- 3.1 Insert into province_names (4 original + 2 new = 6 records)
INSERT INTO province_names (province_id, province_name) VALUES
('ON', 'Ontario'),
('QC', 'Quebec'),
('BC', 'British Columbia'),
('AB', 'Alberta'),
-- New records:
('NS', 'Nova Scotia'),
('MB', 'Manitoba');

-- 3.2 Insert into doctors (3 original + 3 new = 6 records)
INSERT INTO doctors (first_name, last_name, specialty) VALUES
('Marcus', 'Welby', 'Cardiology'),
('Jane', 'Doe', 'Neurology'),
('Alex', 'Karew', 'Pediatrics'),
-- New records:
('Chris', 'Green', 'Oncology'),
('Emily', 'White', 'Orthopedics'),
('Robert', 'Black', 'General Practice');

-- 3.3 Insert into patients (4 original + 4 new = 8 records)
INSERT INTO patients (first_name, last_name, gender, birth_date, city, province_id, allergies, height, weight) VALUES
('John', 'Smith', 'M', '1985-06-15', 'Toronto', 'ON', 'Penicillin', 180, 85),
('Alice', 'Johnson', 'F', '1998-11-20', 'Vancouver', 'BC', 'None', 165, 62),
('Michael', 'Williams', 'M', '1960-03-01', 'Montreal', 'QC', 'Latex', 175, 95),
('Sarah', 'Brown', 'F', '2005-09-10', 'Calgary', 'AB', 'Pollen', 155, 50),
-- New records with variations:
('Linda', 'Lee', 'F', '1975-01-25', 'Halifax', 'NS', NULL, 168, 70), -- NULL for allergies, new province
('David', 'Wong', 'M', '2010-12-05', 'Winnipeg', 'MB', 'Peanuts', 145, 45), -- Child, new province
('Olivia', 'Garcia', 'F', '1990-04-18', 'Edmonton', 'AB', 'Dust', 170, 68), -- Different city in existing province
('Ethan', 'Ng', 'O', '1955-07-22', 'Toronto', 'ON', 'Kiwi', 178, 88); -- 'O' for gender

-- 3.4 Insert into admissions (4 original + 4 new = 8 records)
INSERT INTO admissions (patient_id, admission_date, discharge_date, diagnosis, attending_doctor_id) VALUES
(1, '2023-10-01', '2023-10-05', 'Hypertension', 1),
(2, '2023-10-10', '2023-10-10', 'Migraine', 2),
(3, '2023-11-05', NULL, 'Stroke', 2), -- Still admitted (discharge_date is NULL)
(4, '2023-11-08', '2023-11-09', 'Flu', 3),
-- New records:
(5, '2023-11-20', '2023-11-22', 'Pneumonia', 4), -- Uses new doctor ID
(6, '2023-11-25', NULL, 'Fractured Leg', 5), -- New patient and doctor, still admitted
(7, '2023-12-01', '2023-12-01', 'Fever', 6), -- Same-day admission/discharge
(8, '2023-12-05', '2023-12-10', 'Checkup', 6); -- Routine or non-acute admission


----------------------------------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------------------------------

/*
Q1:
Show unique birth years from patients and order them by ascending.
*/

-- Ans1:
-- Method 1:
SELECT DISTINCT year(birth_date) AS birth_year
FROM patients
ORDER BY birth_year ASC;

----------------------------------------------------------------------------------------------------------------------------

/*
Q2:
Show unique first names from the patients table which only occurs once in the list.

For example, if two or more people are named 'John' in the first_name column then don't include their name in the output list. If only 1 person is named 'Leo' then include them in the output.
*/

-- Ans2:
-- Method 1:
SELECT first_name
FROM patients
group by first_name
having count(*) = 1;

----------------------------------------------------------------------------------------------------------------------------

/*
Q3:
Show first name, last name, and gender of patients whose gender is 'M'
*/

-- Ans3:
-- Method 1:
select first_name, last_name, gender
from patients
where gender = 'M';

----------------------------------------------------------------------------------------------------------------------------

/*
Q4:
Show patient_id and first_name from patients where their first_name start and ends with 's' and is at least 6 characters long.
*/

-- Ans4:
-- Method 1:
select patient_id, first_name
from patients
where len(first_name) >= 6
and lower(first_name) like 's%s';

----------------------------------------------------------------------------------------------------------------------------

/*
Q5:
Show patient_id, first_name, last_name from patients whos diagnosis is 'Dementia'.
Primary diagnosis is stored in the admissions table.
*/

-- Ans5:
-- Method 1:
select p.patient_id, p.first_name, p.last_name
from patients as p 
Left join admissions as a 
using(patient_id)
where diagnosis = 'Dementia';


----------------------------------------------------------------------------------------------------------------------------

/*
Q6:
Display every patient's first_name.
Order the list by the length of each name and then by alphabetically.
*/

-- Ans6:
-- Method 1:
select first_name
from patients
order by length(first_name), first_name;

----------------------------------------------------------------------------------------------------------------------------

/*
Q7:
Show the total amount of male patients and the total amount of female patients in the patients table.
Display the two results in the same row.
*/

-- Ans7:
-- Method 1:
select 
SUM(
case 
when gender = 'M' then 1
end) as male_count,

sum(
case
when gender = 'F' then 1
end) as female_count

from patients;

----------------------------------------------------------------------------------------------------------------------------

/*
Q8:
Show first and last name, allergies from patients which have allergies to either 'Penicillin' or 'Morphine'. Show results ordered ascending by allergies then by first_name then by last_name.
*/

-- Ans8:
-- Method 1:
select first_name, last_name, allergies
from patients
where allergies in ('Penicillin', 'Morphine')
order by allergies, first_name, last_name;

----------------------------------------------------------------------------------------------------------------------------

/*
Q9:
Show patient_id, diagnosis from admissions. Find patients admitted multiple times for the same diagnosis.
*/

-- Ans8:
-- Method 1:
select p.patient_id, a.diagnosis
from patients as p 
join admissions as a 
using(patient_id)
group by p.patient_id, a.diagnosis
having count(a.diagnosis) > 1;

----------------------------------------------------------------------------------------------------------------------------

/*
Q10:
Show the city and the total number of patients in the city.
Order from most to least patients and then by city name ascending.
*/

-- Ans10:
-- Method 1:
select city, count(*) as total_patients
from patients 
group by city
order by count(*) desc, city asc;

----------------------------------------------------------------------------------------------------------------------------

/*
Q11:
Show first name, last name and role of every person that is either patient or doctor.
The roles are either "Patient" or "Doctor"
*/

-- Ans11:
-- Method 1:
select first_name, last_name, 'Patient' as role
from patients 
union all 
select first_name, last_name, 'Doctor' as role
from doctors;

----------------------------------------------------------------------------------------------------------------------------

/*
Q12:
Show first name and last name of patients who does not have allergies. (null)
*/

-- Ans12:
-- Method 1:
select first_name, last_name
from patients 
where allergies is null;

----------------------------------------------------------------------------------------------------------------------------

/*
Q13:
Show first name of patients that start with the letter 'C'
*/

-- Ans13:
-- Method 1:
select first_name
from patients 
where first_name like 'C%';

----------------------------------------------------------------------------------------------------------------------------

/*
Q14:
Show all allergies ordered by popularity. Remove NULL values from query.
*/

-- Ans14:
-- Method 1:
select allergies, count(allergies)
from patients 
where allergies is not null
group by allergies
order by count(allergies) desc;

----------------------------------------------------------------------------------------------------------------------------

/*
Q15:

*/

-- Ans15:
-- Method 1:
-- Test













